In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

In [2]:
torch.set_printoptions(precision=10)

In [3]:
plikembd = open("../embeddings_small.txt")

embeddingscuda_chars = {}
embeddingscuda_chars['🈳'] = 0

embeddingscuda = torch.tensor([]).to("cuda:0")
embeddingscuda =  torch.cat((embeddingscuda, torch.zeros((400)).to("cuda:0")), 0)

print(embeddingscuda.shape)

for cnt, line in enumerate(plikembd.readlines()):
    exp1 = line.replace("\n", "").split(":")
    
    znak = exp1[0]
    embedding = torch.tensor([float(liczba) for liczba in exp1[1].replace(" ", "").replace("[", "").replace("]", "").split(",\t")[:-1]]).to("cuda:0")
    
    embeddingscuda_chars[znak] = cnt+1
    embeddingscuda = torch.cat((embeddingscuda, embedding), 0)

embeddingscuda = embeddingscuda.reshape(82, -1)

print(embeddingscuda.shape)
print(embeddingscuda[embeddingscuda_chars['a']])

torch.Size([400])
torch.Size([82, 400])
tensor([-0.1842972338,  0.1992288828, -0.1093217134,  0.0635560006,
        -0.0170881711, -0.0069951965,  0.1790032834, -0.1581476331,
         0.0322009474, -0.1496459693, -0.1595581025, -0.2537648678,
         0.2232226729, -0.1215698719,  0.0059251408, -0.1644940972,
         0.0264174044,  0.0696226805, -0.0749481246,  0.2048900872,
        -0.0481871516, -0.1348041445,  0.1884992272, -0.0314494744,
         0.1081886366,  0.0045932876, -0.1269595921,  0.1936095357,
        -0.0679676980,  0.1462467760, -0.2293221653, -0.2348391116,
        -0.1658135504, -0.0536651835,  0.0345649682, -0.0555758253,
         0.1800168753, -0.1355556846,  0.0986127034, -0.0864410177,
        -0.1333196312, -0.0306926053,  0.2023293376,  0.1658294201,
         0.1575038880,  0.1013869643, -0.0837135166, -0.0609861165,
        -0.1243505403, -0.0618062057, -0.0087015610, -0.1109482422,
        -0.1424127370,  0.1705076247, -0.0412648059, -0.1866530329,
        

In [4]:
polskie = open("../100k.txt", encoding='utf8')
slowa = [slowo.replace("\n", "") for slowo in polskie.readlines()]
polskie.close()
print(slowa[:100])
print(len(slowa))
slowa = slowa[:5000]

['abmicro', '527579', 'agro2011', 'nakamichi', 'dugant', '29082009', 'hannah1', 'ma12po', '580813', 'lukasz4', 'kriket', 'werohaze', 'dron14121987', 'pinot1', 'maria59', 'moltrex', 'kilim', 'Zabeczka', '7937', '7maluszek', 'gacek16', 'ruleta1', '17Cyprian', 'maj2006', 'o2skarek', 'anno01', 'gusciol', 'gumis1', 'malaz11', 'watomzon', 'rumianek', 'neoisp', 'zsofie', 'zipera', '141084', 'ultraboy', '110567', 'anbog', 'troyd34', 'roxboro', 'werunia', 'misiokrzysio', 'hydra6', 'grzes10', 'kakigewy', 'filip98', '20021970', 'natka777', 'patricia', 'polisnova', 'miko20', 'prj2501', 'grobian1', '1972ml', 'robo21101967', 'jw17061958', 'gregordj', 'insko2000', 'maclear', 'zl060374', 'asia7', 'misiu777', 'varius2', 'lgflatron', 'rele1', '130178', 'centrala', 'filoman', '41karat2', '84jaysinner', 'agatina', 'ambasada', 'maly17', 'goszkam', '1branka', 'dasti', 'petro111', '070165', '20111965', 'triforce', 'bogumil1', 'anna3005', 'dron10011001', 'santaclara', 'pas2526', '11011975', 'rest12', 'j87neo'

In [5]:
longestword = 0

unikalneznaki = []
for slowo in slowa:
    for literka in slowo:
        if literka not in unikalneznaki:
            unikalneznaki.append(literka)
    if len(slowo) > longestword:
        longestword = len(slowo)
            
print(unikalneznaki)
print(longestword)
print(len(unikalneznaki))

['a', 'b', 'm', 'i', 'c', 'r', 'o', '5', '2', '7', '9', 'g', '0', '1', 'n', 'k', 'h', 'd', 'u', 't', '8', 'p', '3', 'l', 's', 'z', '4', 'e', 'w', 'x', 'Z', '6', 'C', 'y', 'j', 'f', 'v', 'F', 'P', 'R', 'O', 'T', 'A', 'X', 'L', 'E', 'S', 'I', 'K', 'N', 'D', 'M', 'q', 'H', 'U', 'J', 'Q', 'G', 'W', 'B', 'V', 'Y', '@', '.', '#']
15
65


In [6]:
print(list(embeddingscuda_chars.keys()))
print(len(list(embeddingscuda_chars.keys())))

['🈳', ' ', '!', '#', '$', '%', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
82


In [7]:
onehotvectors = {}
klasyid = {}

for cnt, litera in enumerate(list(embeddingscuda_chars.keys())):
    vector = [0 for __ in range(len(embeddingscuda_chars.keys()))]
    vector[cnt] = 1
    onehotvectors[litera] = torch.Tensor(vector).to("cuda:0")
    
    klasyid[litera] = cnt
    
print(len(onehotvectors))
print(len(onehotvectors['🈳']))
print(len(klasyid))
print(klasyid)

82
82
82
{'🈳': 0, ' ': 1, '!': 2, '#': 3, '$': 4, '%': 5, '*': 6, '+': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ';': 22, '=': 23, '@': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '^': 53, '_': 54, '`': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81}


In [8]:
print(onehotvectors['🈳'])
print(len(onehotvectors['🈳']))

dlugoschotvectora = len(onehotvectors['🈳'])

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
82


In [9]:
print("ilosc wejsc dyskryminatora: " + str(dlugoschotvectora * longestword))
print("ilosc wyjsc generatora: " + str(len(embeddingscuda[0]) * longestword))

ilosc wejsc dyskryminatora: 1230
ilosc wyjsc generatora: 6000


In [10]:
dataset_real = torch.Tensor([]).to("cuda:0")
iloscslow = 0

pusty = torch.zeros(len(embeddingscuda_chars.keys())).to("cuda:0")

for cnt in tqdm(range(len(slowa))):
    slowo = slowa[cnt]
    for index in range(longestword):
        if index<len(slowo):
            dataset_real = torch.cat((dataset_real, onehotvectors[slowo[index]]), 0)
        else:
            dataset_real = torch.cat((dataset_real, pusty), 0)
    iloscslow = iloscslow + 1

dataset_real = dataset_real.reshape(iloscslow, -1)

print(dataset_real.shape)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:16<00:00, 294.24it/s]


torch.Size([5000, 1230])


In [11]:
# vectors = torch.Tensor([
#            [embeddingscuda[embeddingscuda_chars['a']], 
#            embeddingscuda[embeddingscuda_chars['b']], 
#            embeddingscuda[embeddingscuda_chars['c']], 
#            embeddingscuda[embeddingscuda_chars['d']], 
#            embeddingscuda[embeddingscuda_chars['e']],
#            embeddingscuda[embeddingscuda_chars['f']],
#            embeddingscuda[embeddingscuda_chars['g']],
#            embeddingscuda[embeddingscuda_chars['h']]],
#            [embeddingscuda[embeddingscuda_chars['a']], 
#            embeddingscuda[embeddingscuda_chars['b']], 
#            embeddingscuda[embeddingscuda_chars['c']], 
#            embeddingscuda[embeddingscuda_chars['d']], 
#            embeddingscuda[embeddingscuda_chars['e']],
#            embeddingscuda[embeddingscuda_chars['f']],
#            embeddingscuda[embeddingscuda_chars['g']],
#            embeddingscuda[embeddingscuda_chars['h']]],
#            [embeddingscuda[embeddingscuda_chars['a']], 
#            embeddingscuda[embeddingscuda_chars['b']], 
#            embeddingscuda[embeddingscuda_chars['c']], 
#            embeddingscuda[embeddingscuda_chars['d']], 
#            embeddingscuda[embeddingscuda_chars['e']],
#            embeddingscuda[embeddingscuda_chars['f']],
#            embeddingscuda[embeddingscuda_chars['g']],
#            embeddingscuda[embeddingscuda_chars['h']]],
#            [embeddingscuda[embeddingscuda_chars['a']], 
#            embeddingscuda[embeddingscuda_chars['b']], 
#            embeddingscuda[embeddingscuda_chars['c']], 
#            embeddingscuda[embeddingscuda_chars['d']], 
#            embeddingscuda[embeddingscuda_chars['e']],
#            embeddingscuda[embeddingscuda_chars['f']],
#            embeddingscuda[embeddingscuda_chars['g']],
#            embeddingscuda[embeddingscuda_chars['h']]],
#            [embeddingscuda[embeddingscuda_chars['a']], 
#            embeddingscuda[embeddingscuda_chars['b']], 
#            embeddingscuda[embeddingscuda_chars['c']], 
#            embeddingscuda[embeddingscuda_chars['d']], 
#            embeddingscuda[embeddingscuda_chars['e']],
#            embeddingscuda[embeddingscuda_chars['f']],
#            embeddingscuda[embeddingscuda_chars['g']],
#            embeddingscuda[embeddingscuda_chars['h']]],
#            [embeddingscuda[embeddingscuda_chars['a']], 
#            embeddingscuda[embeddingscuda_chars['b']], 
#            embeddingscuda[embeddingscuda_chars['c']], 
#            embeddingscuda[embeddingscuda_chars['d']], 
#            embeddingscuda[embeddingscuda_chars['e']],
#            embeddingscuda[embeddingscuda_chars['f']],
#            embeddingscuda[embeddingscuda_chars['g']],
#            embeddingscuda[embeddingscuda_chars['h']]]
# ]).cuda()

In [12]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()     
        
        self.fc_in = nn.Linear(6000, 4000)
        self.fc_h1 = nn.Linear(4000, 2000)
        self.fc_h2 = nn.Linear(2000, 1000)
        self.fc_h3 = nn.Linear(1000, 200)
#         self.fc_h4 = nn.Linear(100, 50)
        self.fc_h5 = nn.Linear(200, 20)
        self.fc_out = nn.Linear(20, 1)
        
    def forward(self, x):      
        x = self.fc_in(x)
        x = F.leaky_relu(x)
        
        x = self.fc_h1(x)
        x = F.leaky_relu(x)
        
        x = self.fc_h2(x)
        x = F.leaky_relu(x)
        
        x = self.fc_h3(x)
        x = F.leaky_relu(x)
        
#         x = self.fc_h4(x)
#         x = F.relu(x)
        
        x = self.fc_h5(x)
        x = F.leaky_relu(x)
        
        x = self.fc_out(x)
        x = F.sigmoid(x)
        
        return x

In [13]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.fc_in = nn.Linear(500, 800)
        self.fc_h1 = nn.Linear(800, 3500)
#         self.fc_h2 = nn.Linear(1200, 2000)
#         self.fc_h3 = nn.Linear(2000, 2800)
#         self.fc_h4 = nn.Linear(2800, 3500)
        self.fc_out = nn.Linear(3500, 6000)
        
        self.activation1 = nn.PReLU()
        
    def forward(self, x):
        x = self.fc_in (x)
        x = F.leaky_relu(x)
        
        x = self.fc_h1(x)
        x = F.leaky_relu(x)
        
#         x = self.fc_h2(x)
#         x = F.relu(x)
        
#         x = self.fc_h3(x)
#         x = F.relu(x)
        
#         x = self.fc_h4(x)
#         x = F.relu(x)
        
        x = self.fc_out(x)
        x = F.leaky_relu(x)
        
        return x

In [14]:
discriminator = Discriminator().to("cuda:0")
generator = Generator().to("cuda:0")

TensorReal = dataset_real

TensorFakeOuts = torch.zeros(len(dataset_real)).to("cuda:0")
TensorRealOuts = torch.ones(len(dataset_real)).to("cuda:0")

TensorOuts = torch.cat((TensorFakeOuts, TensorRealOuts), 0).to("cuda:0")

In [15]:
TensorReal.shape

torch.Size([5000, 1230])

In [16]:
criterion = nn.BCELoss()
criterion_segment = nn.CrossEntropyLoss()

In [17]:
TensorFakeSeeds = torch.rand(iloscslow, 500).to("cuda:0")
TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")

In [18]:
TensorFakeIns.reshape(100, -1, 400).shape

torch.Size([100, 750, 400])

In [19]:
# def cuda(vectors):    
#     strings = []
#     wyniki_loss = []
#     onehots = torch.tensor([]).to("cuda:0")
    
#     print(vectors.shape, embeddingscuda.shape)
    
#     s1 = torch.sum(vectors**2, dim=1, dtype=torch.float64).to("cuda:0")
#     s2 = torch.sum(embeddingscuda**2, dim=1, dtype=torch.float64).to("cuda:0")
    
#     print(s1.shape, s2.shape)
    
#     sqrted_s1 = torch.sqrt(vectors)
#     sqrted_s2 = torch.sqrt(s2)
    
#     print(sqrted_s1.shape, sqrted_s2.shape)

#     dzielenie = sqrted_s1*sqrted_s2
    
#     for cnt in tqdm(range(len(vectors))):

#         string = ""

#         for cnt2, single in enumerate(vectors[cnt]):
            
#             sumaup = torch.sum(vectors[cnt][cnt2] * embeddingscuda, dim=1, dtype=torch.float64).to("cuda:0")

#             wynik = sumaup/(sqrted[cnt][cnt2].reshape(400, -1)*torch.sqrt(s2))
#             wynik[0] = 0
#             string=string+list(embeddingscuda_chars.keys())[torch.argmax(wynik).item()]
#             wyniki_loss.append(torch.max(wynik).item())
#             onehots = torch.cat((onehots, onehotvectors[list(embeddingscuda_chars.keys())[torch.argmax(wynik).item()]]), 0)
            
#         strings.append(string)
        
#     return strings, onehots.reshape(len(vectors), -1), np.mean(np.array(wyniki_loss))
# # %time cuda(vectors)

In [20]:
# def cuda2(vectors):
#     print(vectors.shape, embeddingscuda.shape)

#     sqrted_sum_s1 = torch.sqrt(torch.sum(vectors**2, dim=2))
#     sqrted_sum_s2 = torch.sqrt(torch.sum(embeddingscuda**2, dim=1))
    
#     print(sqrted_sum_s1.shape, sqrted_sum_s2.shape)
    
#     for vector in embeddingscuda:
    
    
#     return "", "", ""

In [21]:
# translated1, translated2, translated_loss = cuda2(TensorFakeIns.reshape(100, -1, 400))
# print(translated_loss)

In [22]:
# print(translated1[0:20])

In [23]:
discriminator(TensorFakeIns)

C:\Anaconda3\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.5044869184],
        [0.5044723153],
        [0.5044777989],
        ...,
        [0.5044805408],
        [0.5044566989],
        [0.5044723153]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [24]:
optimizerGenerator = optim.Adam(generator.parameters(), lr=1e-3)
optimizerClassifier = optim.Adam(discriminator.parameters(), lr=1e-3)

In [25]:
for epoch in range(1000):
    TensorFakeSeeds = torch.rand(iloscslow, 500).to("cuda:0")
    TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")
    
    TensorIns = torch.cat((TensorFakeIns.detach(), TensorReal), 0).to("cuda:0")
    
     ## DISCRIMINATOR
    discriminator.train()
    optimizerClassifier.zero_grad()
    
    y_ = discriminator(TensorIns)
    lossDiscriminator = criterion(y_, TensorOuts)
    
    lossDiscriminator.backward()
    optimizerClassifier.step()
    
    
    ## GENERATOR
    generator.train()
    optimizerGenerator.zero_grad()
    
    y_ = discriminator(TensorFakeIns)
    lossGenerator = criterion(y_, TensorRealOuts)
    
    lossGenerator.backward()
    optimizerGenerator.step()
    
    #print(xx)
    print(lossDiscriminator, lossGenerator)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 1230 and 6000 in dimension 1 at c:\a\w\1\s\tmp_conda_3.7_104535\conda\conda-bld\pytorch_1550400486030\work\aten\src\thc\generic/THCTensorMath.cu:83